In [ ]:
# Basics of Thermodynamics

We are going to be studying the basic thermodynamics of 3 types of gas molecules today: H$_2$, O$_2$ and H$_2$O. We have provided the structures for you in files called: `h2.xyz`, `o2.xyz` and `h2o.xyz`. You can read about the functions we are going to be using on the ASE wiki.

On vibrations: https://wiki.fysik.dtu.dk/ase/ase/vibrations/vibrations.html

On thermochemistry: https://wiki.fysik.dtu.dk/ase/ase/thermochemistry/thermochemistry.html

Let's first try and visualize them

In [2]:
!ase gui h2o.xyz  # Change the filename to view the other files

# Let's calculate the vibrational frequencies, and other thermodynamic properties. Try to understand what is happening in the script below, and modify which gas is being calculated. Remember the change the filename in the first line, as well as the molecule type.

### Tasks
* Relax geometry
* Calculate vibrational frequencies
* Using the ase gui
    * Look at the relaxation of the atoms
    * Look at the vibrational modes
* Estimate zero point vibrational energy, heat capacity, entropic contributions in different temperatures
    * Hint: Look in the output files from the vibrational calculation logs
* Calculate when thermal dissociation of water into O$_2$ and H$_2$ becomes feasible, e.g. using excel

In [1]:
%%writefile h2o_free_en_gas.py
import numpy as np

from ase.io import read
from ase.thermochemistry import IdealGasThermo
from ase.optimize import BFGS
from ase.visualize import view
from ase.vibrations import Vibrations
from ase.parallel import paropen, parprint

from gpaw import GPAW, FermiDirac

molecule = 'h2o'  # Change this depending on gas type, 'h2', 'o2' or 'h2o'

system = read(molecule+'.xyz') # Read the atoms object

# Help GPAW with convergence
if molecule == 'h2o':
    system.set_initial_magnetic_moments(magmoms=[0.3, 0.3])

system.center(vacuum=5)  # shouldn't be too small (avoid neighbour interaction)

calc = GPAW(txt=molecule+'.txt',
            xc='PBE', # what is the effect of xc functional
            h=0.2)   # smaller grid leads to more accurate calculation

system.set_calculator(calc)
opt = BFGS(system, trajectory=molecule+'.traj')
opt.run(fmax=0.005) # how well relaxed in terms of forces
calc.write(molecule+'.gpw', 'all')

en = system.get_potential_energy()

parprint('Molecule: {}\tEnergy: {:.3f} eV'.format(molecule, en))

vib = Vibrations(system, name=molecule+'_vib',
                 nfree=2, delta=0.01) # approximation method and displacement
vib.run()
vib_energies = vib.get_energies()

parprint(vib_energies)
vib.summary()

for i in range(3*len(system)): # based on the number of atoms: 3N
    vib.write_mode(i)
    
# Settings for the IdealGasThermo function
thermosettings = {'o2': {'geometry': 'linear', 'spin': 1.0, 'symmetrynumber': 2},
                  'h2': {'geomet1 asry': 'linear', 'spin': 0.0, 'symmetrynumber': 2},
                  'h2o': {'geometry': 'nonlinear', 'spin': 0.0, 'symmetrynumber': 2}}

thermo = IdealGasThermo(vib_energies=vib_energies,
                        potentialenergy=en,
                        atoms=system,
                        **thermosettings[molecule]  # The ** is dictionary unpacking
                        )

filename  = molecule + '_free.txt'  # output file name
with paropen(filename,  print:\t)'w') as fid:
    print('Molecule:', molecule, file=fid)
    for T in np.linspace(200., 6000., 50):
        G = thermo.get_gibbs_energy(temperature=T, pressure=101325.)
        print('Temp: {:.3f} K  \tGibbs Free Energy: {:.4f} eV '.format(T, G), file=fid)

Overwriting h2o_free_en_gas.py


In [2]:
!qsub.py -p 8 -t 1 h2o_free_en_gas.py

7247126.hnode2


In [4]:
!qstat -u $USER


hnode2: 
                                                                                  Req'd    Req'd       Elap
Job ID                  Username    Queue    Jobname          SessID  NDS   TSK   Memory   Time    S   Time
----------------------- ----------- -------- ---------------- ------ ----- ------ ------ --------- - ---------
7246940.hnode2          s171629     hpc      C6H6.py           28291     1      4    --   02:00:00 R  01:08:44


Read the resulting output files once they are done

In [ ]:
# Adjust the name to the filename you want to read
!cat "$(ls -t o2_free_en_gas.py.e* | head -1)"  # Only read the newest file with the specified name format

In [ ]:
# Read all of the files called something with '_free.txt', i.e. h2_free.txt, o2_free.txt and h2o_free.txt
!tail -n +1 *_free.txt